# Modeule 2: Electricity Retailer Problem and Risk Measures

**Petrobras Course on Stochastic Optimization**\
Instructors: Steve Gabriel \& Dominic Flocco\
Date: October 2024

## Step 0: Install Code Dependencies

Before we get started, run the following cells to load the necessary Python packages for the notebook and some helper functions we'll use to display model solutions.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyomo.environ import *
from pyomo.mpec import *
pd.set_option('display.float_format', '{:.2f}'.format)

# set solver
opt = SolverFactory('glpk')

In [ ]:
# helper function to comput profit in each scenario
def scenario_profit(m,s):

    # revenue from selling energy
    revenue = sum(m.electricity_price*m.demand[t] for t in m.periods)

    # cost of purchasing contracts
    contract_cost = 0
    for t in m.periods:
        contract_cost += sum(m.contract_price[c]*m.contract_purchase[c].value for c in m.contracts)

    # cost of pool purchases
    pool_cost = sum(m.pool_price[t,s]*m.pool_purchase[t,s].value for t in m.periods)

    return revenue - (contract_cost + pool_cost)

# helper function for plotting cummulative distribution function
def plot_cdf(m, plot_title=""):
    expected_scenario_profits = {s: scenario_profit(m,s) for s in m.scenarios}
    expected_profit_df = pd.DataFrame.from_dict(expected_scenario_profits,
                                            orient='index',columns=['Profit ($)'])
    expected_profit_df.index.name = 'Scenario'
    data = expected_profit_df['Profit ($)'].to_numpy()
    sorted_data = np.sort(data)
    probabilities = np.arange(1,len(sorted_data) + 1)/len(sorted_data)
    expected_profit = np.mean(sorted_data)
    plt.step(sorted_data, probabilities, where='post',color='black')
    plt.plot(sorted_data, probabilities, 'ko')

    plt.axvline(x=expected_profit,color='r',linestyle='--')
    plt.text(x=expected_profit+50,y=0.95,s='Expected Profit', fontsize=12,color='r',va='baseline',ha='left')
    plt.ylim([0,1])
    plt.xlabel('Profit ($)')
    plt.ylabel('Probability')
    plt.title(f'CDF for {plot_title}')
    plt.grid()


## Problem Description
Adapted from: Conejo, A. J., Carrión, M., & Morales, J. M. (2010). *Decision making under uncertainty in electricity markets* (Vol. 1, pp. 376-384). New York: Springer. Chapter 4.

### Problem Statement
Consider the problem faced by an elextricity retailer that seeks to determin the purchases in the futures market in order to maximize the profit resulting from selling energy to a group of clients. In doing so, the retailer buys energy in both the pool and a futres market. The price of the energy in the pool in each period $t$ is assumed to be unkonwn and is characterized as a random variable. The retailer participates in the futures market by buying energy through three different contracts, defined by a purchasing price, and a maximum quantity of power that can be purchased. The demand of the consumers in each period is assumed to be known, while the selling price of energy to the clients is fixed to $35/MWh. For this example, we consider a planning ohrizon comprised of 3 hourly periods.

The client demands for the three periods are 150, 225, and 175 MW. The data concerning the forward contracts available in the futures market are given in the table below:

| Contract | Price ($/MWh) | Maximum quantity (MW) |
| -------- | ------------- | --------------------- |
| 1 | 24 | 50 |
| 2 | 25 | 30 |
| 3 | 36 | 25 |

The price of the electricity in the pool in every period is represented by a set of 10 equiprobablt scenarios. Pool price data is provided in the table below:

| Scenario | Period 1 | Period 2 | Period 3 |
| - | - | - | - |
| 1 | 28.5 | 36.3 | 31.4 |
| 2 | 27.3 | 37.5 | 29.6 |
| 3 | 29.4 | 35.7 | 31.3 |
| 4 | 33.9 | 35.4 | 35.1 |
| 5 | 34.5 | 38.9 | 37.5 |
| 6 | 29.2 | 34.8 | 31.2 |
| 7 | 34.1 | 36.9 | 35.4 |
| 8 | 33.4 | 35.4 | 34.9 |
| 9 | 28.4 | 36.3 | 32.9 |
| 10 | 27.6 | 38.9 | 32.1 |

The decision-making problem can be formulated as a two-stage stochastic program with recourse.

### Problem Formulation
First, define the following variables and data:
- $x_f$ for $f = 1,2,3$: power puchased under contract $f$
- $y_{t\omega}$ for $t = 1,2,3$ and $\omega = 1, \ldots, 10 $: power purchased from the pool in period $t$ and scenario $\omega$
- $\lambda_\omega^P$ for $\omega = 1, \ldots, 10 $: price of energy in the pool in scenario $\omega$
- $\lambda_f^F$ for $f = 1,2,3$: purchase price of contract $f$
- $X_f^\text{max}$ for $f = 1,2,3$: maximum quantity of power that can be purchased under contract $f$
- $P_t^C$ for $t = 1,2,3$ demand of the consumers in period $t$
- $\lambda^C$: fixed selling price of energy to to clients
- $\pi_\omega$ for $\omega = 1, \ldots, 10 $: probability of scenario $\omega$

The risk-neutral profit maximization problem is

$$
\begin{align*}
\max \quad & \sum_{t=1}^3 \lambda^C P_t^C - \sum_{f=1}^3 \sum_{t=1}^3 \lambda_f^Fx_f - \sum_{\omega = 1}^{10} \pi_\omega \sum_{t=1}^3 \lambda_{t\omega}^Py_{t\omega} \\
\text{s.t.}  \quad & 0 \leq x_f \leq X_f^\text{max}, \qquad f = 1,2,3 & \text{Max contract quantity}\\
& \sum_{f=1}^3 x_f + y_{t\omega} = P_t^C, \qquad t = 1,2,3; \omega = 1,\ldots, 10 & \text{Demand satisfaction}\\
& y_{t\omega} \geq 0, \qquad t = 1,2,3; \omega = 1,\ldots, 10 & \text{Nonnegativity}
\end{align*}
$$

To begin, run the cells below to solve the risk-neutral profit maximization model.

In [ ]:
# store model data

num_contracts = 3
num_periods = 3
num_scenarios = 10

contract_prices = {1: 34, 2: 35, 3: 36}
max_contract_quantities = {1: 50, 2: 30, 3: 25}
pool_prices = np.array([[28.5, 36.3, 31.4], # Scenario 1
                        [27.3, 37.5, 29.6], # Scenario 2
                        [29.4, 35.7, 31.3], # Scenario 3
                        [33.9, 35.4, 35.1], # Scenario 4
                        [34.5, 38.9, 37.5], # Scenario 5
                        [29.2, 34.8, 31.2], # Scenario 6
                        [34.1, 36.9, 35.4], # Scenario 7
                        [33.4, 35.4, 34.9], # Scenario 8
                        [28.4, 36.3, 32.9], # Scenario 9
                        [27.6, 38.9, 32.1]]) #Scenario 10
demand = {1: 150, 2: 225, 3: 175}
electricity_price = 35


In [ ]:
# delcare model sets and parameters
model = ConcreteModel(name='Risk Neutral Model')

model.contracts = RangeSet(1, num_contracts)
model.periods = RangeSet(1, num_periods)
model.scenarios = RangeSet(1, num_scenarios)

model.scenario_prob = Param(model.scenarios,
                             initialize={s:1/num_scenarios for s in model.scenarios})
model.contract_price = Param(model.contracts, # $/MWh
                             initialize=contract_prices)
model.max_contract_qty = Param(model.contracts, # MW
                               initialize=max_contract_quantities)
model.pool_price = Param(model.periods, model.scenarios,
                         initialize={(t,s): pool_prices[s-1,t-1]
                                        for t in model.periods
                                            for s in model.scenarios})
model.demand = Param(model.periods,# MW
                     initialize=demand)
model.electricity_price = Param(initialize=electricity_price) # $/MWh

In [ ]:
# declare risk neutral model variables
model.contract_purchase = Var(model.contracts, # MW
                              within=NonNegativeReals)
model.pool_purchase = Var(model.periods, model.scenarios,
                          within=NonNegativeReals)


In [ ]:
# define model constraints

# upper bound on contract purchases for contract c
def contract_maximum_qty_constraint_(m, c):
    return m.contract_purchase[c] <= m.max_contract_qty[c]
model.contract_maximum_qty_constraint = Constraint(model.contracts, rule=contract_maximum_qty_constraint_)

# power balance constraint in period t and scenario s
def power_balance_constraint_(m, t, s):
    return sum(m.contract_purchase[c] for c in m.contracts) + m.pool_purchase[t,s] == m.demand[t]
model.power_balance_constraint = Constraint(model.periods, model.scenarios, rule=power_balance_constraint_)


In [ ]:
# define model objective
def expected_profit_(m):

    # revenue from selling energy
    revenue = sum(m.electricity_price*m.demand[t] for t in m.periods)

    # cost of purchasing contracts
    contract_cost = 0
    for t in model.periods:
        contract_cost += sum(m.contract_price[c]*m.contract_purchase[c] for c in m.contracts)

    # expected cost of pool purchases
    pool_cost = 0
    for s in model.scenarios:
        pool_cost += model.scenario_prob[s] * sum(m.pool_price[t,s]*m.pool_purchase[t,s]
                                                    for t in model.periods)
    return revenue - (contract_cost + pool_cost)

model.expected_profit_obj = Objective(rule=expected_profit_, sense=maximize)



Now, run the cell below to solve the risk-neutral model.

In [ ]:
# solve model
res = opt.solve(model)

# check that solver status exited normally
assert res.solver.status == SolverStatus.ok, \
        f"Solver did not exit normally, termination condition {res.solver.status}."

# visualize model solution
print(f"**** EXPECTED PROFIT = ${model.expected_profit_obj():.2f} ****")
plot_cdf(model, plot_title=model.name)


## Risk-Averse Decision Making
Now, we will analyse various risk measures to explore the affect of risk-averse decision making.

### Shortfall Probability
The shortfall probability is equal to the probability of the profit being less than a pre-fixed value $\eta$. Mathematically the shortfall probability is defined as
$$ \text{SP}(\eta, x) = P(\omega | f(x,\omega)< \eta) \quad \forall \eta \in \mathbb{R}$$
The shortfall probability can be incorporated into the risk-neutral problem by introducing auxillary variable $\theta_\omega\in\{0,1\}$ which equals 1 if there is a shortfall in scenario $\omega$ and 0 otherwise:

$$
\begin{align*}
\max \quad & (1-\beta) \left(\sum_{t=1}^3 \lambda^C P_t^C - \sum_{f=1}^3 \sum_{t=1}^3 \lambda_f^Fx_f- \sum_{\omega = 1}^{10} \pi_\omega \sum_{t=1}^3 \lambda_{t\omega}^Py_{t\omega}\right) \\
& \qquad - \beta \sum_{\omega=1}^{10} \pi_\omega \theta_\omega \\
\text{s.t.}  \quad & 0 \leq x_f \leq X_f^\text{max}, \qquad f = 1,2,3 & \text{Max contract quantity}\\
& \sum_{f=1}^3 x_f + y_{t\omega} = P_t^C, \qquad t = 1,2,3; \omega = 1,\ldots, 10 & \text{Demand satisfaction}\\
& y_{t\omega}, \qquad t = 1,2,3; \omega = 1,\ldots, 10 & \text{Nonnegativity}\\
& \eta - \left(\sum_{t=1}^3 \lambda^C P_t^C - \sum_{f=1}^3 \sum_{t=1}^3 \lambda_f^Fx_f-  \sum_{t=1}^3 \lambda_{t\omega}^Py_{t\omega}\right) \leq M \theta_\omega & \text{Shortfall definition}\\
& \theta_\omega \in \{0,1\}, \qquad \forall \omega = 1,\ldots, 10
\end{align*}
$$
Note that this results in a mixed-integer linear program (MILP). For this example, we set $\eta = \$175$ and $M = 10,000$. Run the cells below to solve the risk-averse shrotfall probability model. How does this change the model solution?


In [ ]:
# create a new model instance
sp_model = model.create_instance()
sp_model.name = 'Shortfall Probability Model'

# define shortfall probability model data
sp_model.weight = Param(initialize=1.0,
                        mutable=True) # beta

sp_model.shortfall_thresh = Param(initialize=175) # eta ($)

sp_model.big_M = Param(initialize=10000)

# define auxiliary variable
sp_model.theta = Var(sp_model.scenarios,
                     within=Binary)

# add shortfall definition constraint
def shortfall_def_constraint_(m, s):

    # revenue from selling energy
    revenue = sum(m.electricity_price*m.demand[t] for t in m.periods)

    # cost of purchasing contracts
    contract_cost = 0
    for t in model.periods:
        contract_cost += sum(m.contract_price[c]*m.contract_purchase[c] for c in m.contracts)

    # expected cost of pool purchases
    pool_cost = sum(m.pool_price[t,s]*m.pool_purchase[t,s] for t in model.periods)

    profit = revenue - (contract_cost + pool_cost)

    return m.shortfall_thresh - profit <= m.big_M*m.theta[s]

sp_model.shortfall_def_constraint = Constraint(sp_model.scenarios, rule=shortfall_def_constraint_)


In [ ]:
# set shortfall probability risk-averse objective
def shortfall_probability_objective_(m):
    expected_profit = expected_profit_(m)

    shortfall_probability = sum(m.scenario_prob[s]*m.theta[s] for s in m.scenarios)

    return (1-m.weight)*expected_profit - m.weight*shortfall_probability

sp_model.shortfall_probability_obj = Objective(rule=shortfall_probability_objective_,
                                               sense=maximize)
# deactivate old objective
sp_model.expected_profit_obj.deactivate()

In [ ]:
# solve model

res = opt.solve(sp_model)

# check that solver status exited normally
assert res.solver.status == SolverStatus.ok, \
        f"Solver did not exit normally, termination condition {res.solver.status}."

# compute shortfall probability
shortfall_prob = sum(sp_model.scenario_prob[s]*sp_model.theta[s].value for s in sp_model.scenarios)
print(f"**** EXPECTED PROFIT = ${sp_model.expected_profit_obj():.2f} ****")
print(f"**** SHORTFALL PROBABILITY = {shortfall_prob:.2f} ****")

# plot CDF
plot_cdf(sp_model, plot_title=fr'{sp_model.name}, $\beta = {sp_model.weight.value}$')
plt.axvline(x=sp_model.shortfall_thresh.value,color='b',linestyle='--')
plt.text(x=sp_model.shortfall_thresh.value-100,y=0.7,s='$\eta$', fontsize=12,color='b',va='baseline',ha='right')
plt.show()


**Exercise 1 (JN-7):** Modify the risk weight parameter $\beta$. Keep in mind that $\beta = 0$ corresponds to the fully risk-neutral case and $\beta = 1$ corresponds to the fully risk-averse case.
* What happens to the electricity producer's expected profit as this weight changes?

In [ ]:
# modify risk weighting
new_beta_value = 1.0 # modify me!

assert (new_beta_value > 0) and (new_beta_value <= 1),\
            "Beta value must be between 0 (exlusive) and 1 (inclusive). Try again!"


Now, run the cell below to solv the risk-averse shortfall probability model to see how this effects the solution. (make sure you run the cell above so your change takes place)

In [ ]:
# create new model instance
sp_model_temp = sp_model.create_instance()

sp_model.weight.value = new_beta_value
# solve model
res = opt.solve(sp_model_temp)

# check that solver status exited normally
assert res.solver.status == SolverStatus.ok, \
        f"Solver did not exit normally, termination condition {res.solver.status}."

# compute shortfall probability
shortfall_prob = sum(sp_model_temp.scenario_prob[s]*sp_model_temp.theta[s].value for s in sp_model_temp.scenarios)
print(f"**** EXPECTED PROFIT = ${sp_model_temp.expected_profit_obj():.2f} ****")
print(f"**** SHORTFALL PROBABILITY = {shortfall_prob:.2f} ****")

plot_cdf(sp_model_temp, plot_title=fr'{sp_model_temp.name}, $\beta = {sp_model_temp.weight.value}$')
plt.axvline(x=sp_model_temp.shortfall_thresh.value,color='b',linestyle='--')
plt.text(x=sp_model_temp.shortfall_thresh.value-100,y=0.7,s='$\eta$', fontsize=12,color='b',va='baseline',ha='right')
plt.show()

Now, we'll conduct a sensitivity analysis on the weight $\beta$ to generalize this trend. Run the cells below to solve the shortfall probability risk-averse model for varying values of $\beta$ and plot the efficient frontier.

In [ ]:
# generate a sequence of weights
betas = np.linspace(0.1,1,100)

# initialize data storing objects
expected_profits = np.zeros_like(betas)
shortfall_probs = np.zeros_like(betas)

# solve the model for each value of beta
for i, beta in np.ndenumerate(betas):

    # create new model instance
    sp_model_temp = sp_model.create_instance()

    # set value of beta
    sp_model_temp.weight.set_value(beta)

    # solve model
    res = opt.solve(sp_model_temp)

    # check that solver status exited normally
    assert res.solver.status == SolverStatus.ok, \
            f"Solver did not exit normally, termination condition {res.solver.status}."

    # store results
    expected_profits[i] = sp_model_temp.expected_profit_obj()
    shortfall_probs[i] = sum(sp_model_temp.theta[s].value for s in sp_model_temp.scenarios)/num_scenarios



In [ ]:
# plot efficient frontier
plt.scatter(expected_profits,shortfall_probs,color='black')
plt.plot(expected_profits,shortfall_probs,color='black',linestyle='--')
plt.ylabel("Shortfall Probability")
plt.xlabel('Expected Profit ($)')
plt.title('Shortfall Probability Efficient Frontier')
plt.grid()
plt.show()

### Value at Risk (VaR)
For a given reliability level $\alpha \in (0,1)$, the value-at-risk (VaR) is equal tot he largest value $\eta$ ensuring that the probability of obtaining a profit less than $\eta$ is lower than $1-\alpha$. Mathematically, the VaR is expressed as
$$ \text{VaR}(\alpha, x) = \max\left\{\eta : P(\omega | f(x, \omega) < \eta ) \leq 1-\alpha \right\}, \quad \forall \alpha \in (0,1).$$
In other words, the VaR is the $(1-\alpha)$-quantile of the profit distribution.

We can in corporate VaR into the risk-neutral problem by introducing $\eta$ as a decision variable:

$$
\begin{align*}
\max \quad & (1-\beta) \left(\sum_{t=1}^3 \lambda^C P_t^C - \sum_{f=1}^3 \sum_{t=1}^3 \lambda_f^Fx_f- \sum_{\omega = 1}^10 \pi_\omega \sum_{t=1}^3 \lambda_{t\omega}^Py_{t\omega}\right) \\
& \qquad + \beta \eta  \\
\text{s.t.}  \quad & 0 \leq x_f \leq X_f^\text{max}, \qquad f = 1,2,3 & \text{Max contract quantity}\\
& \sum_{f=1}^3 x_f + y_{t\omega} = P_t^C, \qquad t = 1,2,3; \omega = 1,\ldots, 10 & \text{Demand satisfaction}\\
& y_{t\omega}, \qquad t = 1,2,3; \omega = 1,\ldots, 10 & \text{Nonnegativity}\\
& \eta - \left(\sum_{t=1}^3 \lambda^C P_t^C - \sum_{f=1}^3 \sum_{t=1}^3 \lambda_f^Fx_f-  \sum_{t=1}^3 \lambda_{t\omega}^Py_{t\omega}\right) \leq M \theta_\omega & \text{Shortfall definition}\\
& \sum_{\omega = 1}^{10} \pi_\omega \theta_\omega \leq 1- \alpha & \text{VaR definition}\\
& \theta_\omega \in \{0,1\}, \qquad \forall \omega = 1,\ldots, 10
\end{align*}
$$
Note that, similar to the shortfall probability model, this results in a MILP formulation. The crucial difference between the VaR formulation and the shortfall probability formulation is that $\eta$ is a decision variable in the VaR model, and data in the shortfall probability problem.

Now, run the cells below to formulate and solve the VaR risk-averse problem. How does the solution differ from the risk-neutral case and shortfall probability risk-averse case?

In [ ]:
# create a new model instance
var_model = sp_model.create_instance()
var_model.name = 'VaR Model'

# initialize reliability level
var_model.alpha = Param(initialize=0.8,
                        mutable=True)

# initialize VaR variable
var_model.eta = Var(within=NonNegativeReals)

# delete unneeded model attributes from shortfall probabilty case
del var_model.shortfall_def_constraint
del var_model.shortfall_thresh

# add VaR defining constraint
def var_def_constraint_(m, s):

    # revenue from selling energy
    revenue = sum(m.electricity_price*m.demand[t] for t in m.periods)

    # cost of purchasing contracts
    contract_cost = 0
    for t in model.periods:
        contract_cost += sum(m.contract_price[c]*m.contract_purchase[c] for c in m.contracts)

    # expected cost of pool purchases
    pool_cost = sum(m.pool_price[t,s]*m.pool_purchase[t,s] for t in model.periods)

    profit = revenue - (contract_cost + pool_cost)

    return m.eta - profit <= m.big_M*m.theta[s]

var_model.var_def_constraint = Constraint(var_model.scenarios, rule=var_def_constraint_)

# add VaR reliability constraint
def reliability_constraint_(m):
    return sum(m.scenario_prob[s]*m.theta[s] for s in m.scenarios) <= 1-m.alpha
var_model.reliability_constraint = Constraint(rule=reliability_constraint_)



In [ ]:
# declare VaR objective
def var_objective_(m):
    expected_profit = expected_profit_(m)

    value_at_risk = m.eta

    return (1-m.weight)*expected_profit + m.weight*value_at_risk

var_model.var_objective = Objective(rule=var_objective_,
                                    sense=maximize)

# deactivate old shortfall probability  objective
var_model.shortfall_probability_obj.deactivate()

In [ ]:
# solve model
res = opt.solve(var_model)

# check that solver status exited normally
assert res.solver.status == SolverStatus.ok, \
        f"Solver did not exit normally, termination condition {res.solver.status}."

# output model results
print(f"**** EXPECTED PROFIT = ${var_model.expected_profit_obj():.2f} ****")
print(f"**** VaR = ${var_model.eta.value:.2f} ****")

# plot CDF
plot_cdf(var_model, plot_title=fr'{var_model.name}, $\beta = {var_model.weight.value}$')
plt.axvline(x=var_model.eta.value,color='b',linestyle='--')
plt.text(x=var_model.eta.value-50,y=0.7,s=r'$VaR(\alpha)$', fontsize=12,color='b',va='baseline',ha='right')
plt.show()
#TODO: Reset beta for VaR example

**Exercise 2 (JN-7)**: Next, modify the risk weight $\beta$. Keep in mind that $\beta = 0$ corresponds to the fully risk-neutral case and $\beta = 1$ corresponds to the fully risk-averse case.
* What happens to the electricity producer's expected profit as this weight changes?

In [ ]:
# modify risk weighting
new_beta_value = 0.2 # modify me!

assert (new_beta_value > 0) and (new_beta_value <= 1),\
            "Beta value must be between 0 (exlusive) and 1 (inclusive). Try again!"


Now, run the cells below to solve the risk-averse VaR model with modified data.

In [ ]:
# create new model instance
var_model_temp = var_model.create_instance()
var_model_temp.weight.set_value(new_beta_value)

# solve model
res = opt.solve(var_model_temp)

# check that solver status exited normally
assert res.solver.status == SolverStatus.ok, \
        f"Solver did not exit normally, termination condition {res.solver.status}."


# output model solution
print(f"**** EXPECTED PROFIT = ${var_model_temp.expected_profit_obj():.2f} ****")
print(f"**** VaR = ${var_model_temp.eta.value:.2f} ****")

# plot CDF
plot_cdf(var_model_temp, plot_title=fr'{var_model.name}, $\beta = {var_model_temp.weight.value}$')
plt.axvline(x=var_model_temp.eta.value,color='b',linestyle='--')
plt.text(x=var_model_temp.eta.value-50,y=0.7,s=r'$VaR(\alpha)$', fontsize=12,color='b',va='baseline',ha='right')
plt.show()

Now, we'll conduct a sensitivity analysis on the weight $\beta$ to generalize this trend. Run the cells below to solve the VaR risk-averse model for varying values of $\beta$ and plot the efficient frontier.

In [ ]:
# generate different values of beta
betas = np.linspace(0.1,1,100)

# initialize result storing structures
expected_profits = np.zeros_like(betas)
values_at_risk = np.zeros_like(betas)

# solve model for different wieghts
for i, beta in np.ndenumerate(betas):

    # create new model instance
    var_model_temp = var_model.create_instance()

    # set new weight
    var_model_temp.weight.set_value(beta)

    # solve model
    res = opt.solve(var_model_temp)

    # check that solver status exited normally
    assert res.solver.status == SolverStatus.ok, \
            f"Solver did not exit normally, termination condition {res.solver.status}."

    # store solution
    expected_profits[i] = var_model_temp.expected_profit_obj()
    values_at_risk[i] = var_model_temp.eta.value



In [ ]:
# plot efficient frontier
plt.scatter(expected_profits,values_at_risk,color='black')
plt.plot(expected_profits,values_at_risk,color='black',linestyle='--')
plt.ylabel("VaR ($)")
plt.xlabel('Expected Profit ($)')
plt.title('VaR Efficient Frontier')
plt.grid()
plt.show()

### Conditional Value-at-Risk
For a given $\alpha \in (0,1)$, the conditional value-at-risk (CVaR) is defined as the expected value of the profit smaller than the $(1-\alpha)$-quantile of the profit distribution. Mathematically, the CVaR for a discrete distribution is defined as
$$ \text{CVaR}(\alpha,x) = \max\left\{ \eta - \frac{1}{1-\alpha} \mathbb{E}_\omega \{\max \eta - f(x,\omega),0\}\right\}$$
The CVaR is also referred to as the mean excess loss or average value-at-risk. We can incorporate CVaR into our model by adding the auxillary variable $s_\omega$:


$$
\begin{align*}
\max \quad & (1-\beta) \left(\sum_{t=1}^3 \lambda^C P_t^C - \sum_{f=1}^3 \sum_{t=1}^3 \lambda_f^Fx_f- \sum_{\omega = 1}^10 \pi_\omega \sum_{t=1}^3 \lambda_{t\omega}^Py_{t\omega}\right) \\
& \qquad + \beta \left(\eta - \frac{1}{1-\alpha}\sum_{\omega = 1}^{10} \pi_\omega s_\omega \right)\\
\text{s.t.}  \quad & 0 \leq x_f \leq X_f^\text{max}, \qquad f = 1,2,3 & \text{Max contract quantity}\\
& \sum_{f=1}^3 x_f + y_{t\omega} = P_t^C, \qquad t = 1,2,3; \omega = 1,\ldots, 10 & \text{Demand satisfaction}\\
& y_{t\omega}, \qquad t = 1,2,3; \omega = 1,\ldots, 10 & \text{Nonnegativity}\\
& \eta - \left(\sum_{t=1}^3 \lambda^C P_t^C - \sum_{f=1}^3 \sum_{t=1}^3 \lambda_f^Fx_f-  \sum_{t=1}^3 \lambda_{t\omega}^Py_{t\omega}\right) \leq s_\omega & \text{CVaR definition}\\
& s_\omega \geq 0 \qquad \forall \omega = 1,\ldots, 10
\end{align*}
$$
Note that the resulting formulation is a linear program, which is an addition computational benefit to the CVaR model. Now, run the cells below to implement and solve the CVaR risk-averse model.


In [ ]:
# create new model instance
cvar_model = var_model.create_instance()
cvar_model.name = 'CVaR Model'

# declare cvar variable
cvar_model.aux = Var(cvar_model.scenarios,
                     within=NonNegativeReals)

# remove unneeded variables and constraints left over from VaR model
del cvar_model.var_def_constraint, cvar_model.reliability_constraint
del cvar_model.shortfall_probability_obj
del cvar_model.theta, cvar_model.big_M

# add CVaR definint constraint
def cvar_def_constraint_(m, s):

    # revenue from selling energy
    revenue = sum(m.electricity_price*m.demand[t] for t in m.periods)

    # cost of purchasing contracts
    contract_cost = 0
    for t in model.periods:
        contract_cost += sum(m.contract_price[c]*m.contract_purchase[c] for c in m.contracts)

    # expected cost of pool purchases
    pool_cost = sum(m.pool_price[t,s]*m.pool_purchase[t,s] for t in model.periods)

    profit = revenue - (contract_cost + pool_cost)

    return m.eta - profit <= m.aux[s]
cvar_model.cvar_def_constraint = Constraint(cvar_model.scenarios, rule=cvar_def_constraint_)

# declare CVaR objective
def cvar_objective_(m):
    expected_profit = expected_profit_(m)

    conditional_value_at_risk = m.eta - sum(m.scenario_prob[s]*m.aux[s] for s in m.scenarios)/(1-m.alpha)

    return (1-m.weight)*expected_profit + m.weight*conditional_value_at_risk

cvar_model.cvar_objective = Objective(rule=cvar_objective_,
                                    sense=maximize)
# deactivate old objective
cvar_model.var_objective.deactivate()


In [ ]:
# reset weight to risk-averse case
cvar_model.weight.set_value(1.0)

# solve model
res = opt.solve(cvar_model)

# check that solver status exited normally
assert res.solver.status == SolverStatus.ok, \
            f"Solver did not exit normally, termination condition {res.solver.status}."

# compute CVaR
cvar = cvar_model.eta.value - sum(cvar_model.scenario_prob[s]*cvar_model.aux[s].value
                                for s in cvar_model.scenarios)/(1-cvar_model.alpha.value)

# print solution
print(f"**** EXPECTED PROFIT = ${cvar_model.expected_profit_obj():.2f} ****")
print(f"**** CVaR = ${cvar:.2f} ****")

# plot CDF
plot_cdf(cvar_model, plot_title=fr'{cvar_model.name}, $\beta = {cvar_model.weight.value}$')
plt.axvline(x=cvar,color='b',linestyle='--')
plt.text(x=cvar-50,y=0.7,s=r'$CVaR(\alpha)$', fontsize=12,color='b',va='baseline',ha='right')
plt.show()

**Exercise 3 (JN-7)** Now, modify the risk weight parameter $\beta$. Keep in mind that $\beta = 0$ corresponds to the fully risk-neutral case and $\beta = 1$ corresponds to the fully risk-averse case.
* What happens to the electricity producer's expected profit as this weight changes?

In [ ]:
# modify risk weighting
new_beta_value = 1.0 # modify me!

assert (new_beta_value > 0) and (new_beta_value <= 1),\
            "Beta value must be between 0 (exlusive) and 1 (inclusive). Try again!"


Now, run the cells below to solve the risk-averse CVaR model with modified data.

In [ ]:
# create a new model instance
cvar_model_temp = cvar_model.create_instance()

# set new value of beta
cvar_model_temp.weight.set_value(new_beta_value) # modify me!

# solve model
res = opt.solve(cvar_model_temp)

# check that solver status exited normally
assert res.solver.status == SolverStatus.ok, \
        f"Solver did not exit normally, termination condition {res.solver.status}."

# compute CVaR
cvar = cvar_model_temp.eta.value - sum(cvar_model_temp.scenario_prob[s]*cvar_model_temp.aux[s].value
                                for s in cvar_model_temp.scenarios)/(1-cvar_model_temp.alpha.value)

# print solution
print(f"**** EXPECTED PROFIT = ${cvar_model_temp.expected_profit_obj():.2f} ****")
print(f"**** CVaR = ${cvar:.2f} ****")

# plot CDF
plot_cdf(cvar_model, plot_title=fr'{cvar_model_temp.name}, $\beta = {cvar_model_temp.weight.value}$')
plt.axvline(x=cvar,color='b',linestyle='--')
plt.text(x=cvar-50,y=0.7,s=r'$CVaR(\alpha)$', fontsize=12,color='b',va='baseline',ha='right')
plt.show()

Now, we'll conduct a sensitivity analysis on the weight $\beta$ to generalize this trend. Run the cells below to solve the CVaR risk-averse model for varying values of $\beta$ and plot the efficient frontier.

In [ ]:
# generate a sequence of weights
betas = np.linspace(0.1,1,100)

# initialize data storing objects
expected_profits = np.zeros_like(betas)
conditional_values_at_risk = np.zeros_like(betas)

# solve the model for each value of beta
for i, beta in np.ndenumerate(betas):

    # create new model instance
    sp_model_temp = sp_model.create_instance()

    # set value of beta
    cvar_model.weight.set_value(beta)

    # solve model
    res = opt.solve(cvar_model)

    # check that solver status exited normally
    assert res.solver.status == SolverStatus.ok, \
            f"Solver did not exit normally, termination condition {res.solver.status}."
    # compute CVaR
    cvar = cvar_model.eta.value - sum(cvar_model.scenario_prob[s]*cvar_model.aux[s].value
                                        for s in cvar_model.scenarios)/(1-cvar_model.alpha.value)

    # store results
    expected_profits[i] = cvar_model.expected_profit_obj()
    conditional_values_at_risk[i] = cvar



In [ ]:
# plot CVaR efficient frontier
plt.scatter(expected_profits,conditional_values_at_risk,color='black')
plt.plot(expected_profits,conditional_values_at_risk,color='black',linestyle='--')
plt.ylabel("CVaR ($)")
plt.xlabel('Expected Profit ($)')
plt.title('CVaR Efficient Frontier')
plt.grid()
plt.show()